In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath(''))
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geo import COM, PROV, REG, DEMOG, find_place
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib notebook

def datech(s):
    return f'{s[6:]}-{s[4:6]}-{s[0:4]}'

def yyymmdd2yyww(yyyymmdd):
    if isinstance(yyyymmdd, str):
        return datetime.strptime(yyyymmdd, '%Y%m%d').strftime('%y-%W')
    elif isinstance(yyyymmdd, (list, tuple)):        
        return [yyymmdd2yyww(_) for _ in yyyymmdd]

In [ ]:
CWD = Path(os.path.abspath(''))

data_dir = CWD / 'protezione_civile/COVID-19'
prov_dir = data_dir /  'dati-province'
reg_dir = data_dir /  'dati-regioni'

In [ ]:
DEMOG[['prov_name', 'population']].groupby('prov_name').sum().query('prov_name=="roma"')
DEMOG.query('prov_name=="roma" and Sesso=="totale"')
pop = DEMOG.pivot_table(index=['prov_name'], columns=['Sesso', 'Stato civile', 'Età'])['population']['totale']['totale'][['totale']].reset_index()
prov = PROV.merge(pop, on='prov_name')
# Region merge with demography
# Trento e Bolzano the outsiders
pop_trentino = pop[pop['prov_name'].apply(lambda s: True if 'autono' in s.lower() else False)]['totale'].sum()
pop = pop.append({'prov_name': 'trentino-alto adige / südtirol', 'totale': pop_trentino}, ignore_index=True)
reg = REG.merge(pop, left_on='reg_name', right_on='prov_name')

# Dati della Protezione Civile
Data from [github.com/pcm-dpc/COVID-19](https://github.com/pcm-dpc/COVID-19).

Questi dati sono da aggiornare giornalmente coi comandi:
```
cd covid-19/protezione_civile/COVID-19
git fetch
git merge
```

## Province

In [ ]:
## Raggruppa tutti i dati della protezione civile in un unica tabella
df = []
prec = lambda s: f'{s:.4f}'
for f in sorted(prov_dir.glob('dpc-covid19-ita-province-202*.csv')):
    tmp_df = pd.read_csv(f, encoding='latin-1')
    tmp_df['day'] = str(f).split('/')[-1].split('-')[-1].rstrip('.csv')
    tmp_df['geometry'] = gpd.points_from_xy(tmp_df['long'], tmp_df['lat'])
    tmp_df['lat_long'] = tmp_df['lat'].apply(prec).astype(str) + '_' + tmp_df['long'].apply(prec).astype(str)
    df.append(tmp_df)
    del tmp_df
DF = pd.concat(df)
del df
DF.loc[DF['denominazione_provincia'] == 'Napoli', 'sigla_provincia'] = 'NA'
DF['week'] = DF['day'].apply(yyymmdd2yyww)

In [ ]:
## Raggruppa tutti i dati della protezione civile in un unica tabella
df = []
prec = lambda s: f'{s:.4f}'
for f in sorted(reg_dir.glob('dpc-covid19-ita-regioni-202*.csv')):
    tmp_df = pd.read_csv(f, encoding='latin-1')
    tmp_df['day'] = str(f).split('/')[-1].split('-')[-1].rstrip('.csv')
    tmp_df['geometry'] = gpd.points_from_xy(tmp_df['long'], tmp_df['lat'])
    tmp_df['lat_long'] = tmp_df['lat'].apply(prec).astype(str) + '_' + tmp_df['long'].apply(prec).astype(str)
    df.append(tmp_df)
    del tmp_df
DFR = pd.concat(df)
del df

In [ ]:
df0 = DF.pivot_table(index='sigla_provincia', columns='day', values='totale_casi')

dates = [_ for _ in df0.columns if '20' in _]
df0 = df0.diff(periods=1, axis=1)  # total cases -> new cases each day
window = 7  # Larghezza della finestra mobile
df = df0.rolling(window=window, axis=1).mean()  # Moving Window
dfc = df0.rolling(window=window, axis=1).sum()  # Moving Window Cumulative
# weeks = [_ for _ in dfw.columns if '-' in _]
# weeks.pop(0)
# weeks.pop(-1)
# l = list(zip(df.columns, yyymmdd2yyww(df.columns.tolist())))
# new_cols = pd.MultiIndex.from_tuples(l, names=['day', 'week'])
# df.columns = new_cols

df['totale_casi'] = df.sum(1)
df['nuovi_casi'] = df[dates[-1]]

dfc['totale_casi'] = dfc.sum(1)
dfc['nuovi_casi'] = dfc[dates[-1]]

In [ ]:
# Terapie Intensive Occupate
ti = DFR.pivot_table(index='codice_regione', columns='day', values='terapia_intensiva').reset_index()
TI = REG.merge(ti, left_on='reg_istat_code_num', right_on='codice_regione')
# TI.plot(dates[-1], legend=True, cmap='Reds')

In [ ]:
# Si assume un numero di 14 terapie intensive disponibili ogni 100k abitanti.
# Allarme se si supera il 40% di occupazione
rpop = prov.groupby('reg_istat_code_num').agg('sum')['totale'].reset_index()
rTI = TI.merge(rpop, on='reg_istat_code_num')
rTI[dates] = rTI[dates].divide(rTI.totale, axis=0) * 100e3 / 14 * 100
# rTI['r'] = rTI[dates[-1]] / rTI['totale'] * 100e3 / 14 * 100
rTI.plot(dates[-1], legend=True, cmap='Reds')

In [ ]:
# Associa i dati al territorio
m = prov.merge(df, left_on='prov_acr', right_on='sigla_provincia')
m['casi_100k'] = m['totale_casi'] / m['totale'] * 100e3
m['var_perc_1day'] = 100 * (m[dates[-1]] - m[dates[-window]]) / m[dates[-window]]
# Cumulative cases (Incidence)
mc = prov.merge(dfc, left_on='prov_acr', right_on='sigla_provincia')
mc[dates] = mc[dates].divide(mc['totale'], axis=0) * 100e3  # nuovi casi ogni 100k abitanti
# mc['nuovi_casi_100k'] = mc['nuovi_casi'] / mc['totale'] * 100000

In [ ]:
# Province Incidence -> Region Incidence
mcr = prov.merge(dfc, left_on='prov_acr', right_on='sigla_provincia').dissolve(by='reg_name', aggfunc='sum')
mcr[dates] = mcr[dates].divide(mcr['totale'], axis=0) * 100e3

In [ ]:
def incidence2zone(x):
    assert x >= 0
    if x <= 50:
        return 'green'
    elif x <= 100:
        return 'yellow'
    elif x <= 250:
        return 'orange'
    elif x > 250:
        return 'red'
mcr['zona'] =  mcr[dates[-1]].apply(incidence2zone)

In [ ]:
zona = mcr.dissolve(by='zona', aggfunc='mean')['geometry']
fig, ax = plt.subplots()
for i in range(len(zona)):
    color = zona.iloc[[i]].index
    zona.iloc[[i]].boundary.plot(ax=ax, edgecolor=color, linewidth=4)

In [ ]:
print(f'Window: {window}')
date_ddmmyyyy = datech(dates[-1])
dpi = 150

fig, ax = plt.subplots(1,2)
plt.suptitle(f'Nuovi casi Covid-19 \n aggiornamento: {date_ddmmyyyy}')
m.plot(column='nuovi_casi', legend=True, cmap='Reds', ax=ax[1])
m.dissolve(by='reg_name', aggfunc='sum').plot(column='nuovi_casi', legend=True, cmap='Reds', ax=ax[0])
ax[0].axis('off')
ax[1].axis('off')
plt.savefig('fig/nuovi_casi.png', dpi=dpi)

fig, ax = plt.subplots(1,2)
plt.suptitle(f'Nuovi casi ogni 100.000 abitanti in {window} giorni \n aggiornamento: {date_ddmmyyyy}')
mc.plot(column=dates[-1], legend=True, cmap='Reds', ax=ax[1])
mcr.plot(column=dates[-1], legend=True, cmap='Reds', ax=ax[0])
# for i in range(len(zona)):
#     color = zona.iloc[[i]].index
#     zona.iloc[[i]].boundary.plot(ax=ax[0], edgecolor=color)#, linewidth=3)
#     zona.iloc[[i]].boundary.plot(ax=ax[1], edgecolor=color)#, linewidth=3)
ax[0].axis('off')
ax[1].axis('off')
plt.savefig('fig/nuovi_casi_100k.png', dpi=dpi)

# m.plot(column='var_perc_1day', legend=True, cmap='seismic')
# plt.title(f'Nuovi casi rispetto alla finestra ({window}) precedente (%) \n aggiornamento: {date_ddmmyyyy}')
# plt.axis('off')
# plt.savefig('fig/delta.png', dpi=dpi)

# Andamento per Regioni

In [ ]:
df = DF.pivot_table(index='denominazione_regione', columns='data', values='totale_casi', aggfunc=np.sum)
df['incremento'] = df[df.columns[-1]] - df[df.columns[-2]]
df = df.sort_values(['incremento', df.columns[-1]], ascending=[0, 0])
display(df[df.columns[-3:]])

# Andamento per Province

In [ ]:
df = DF.pivot_table(index=['sigla_provincia', 'denominazione_provincia'], columns='data', values='totale_casi', aggfunc=np.sum)
df['incremento'] = (df.iloc[:, -1] - df.iloc[:, -2])
df = df.sort_values(['incremento', df.columns[-2]], ascending=[0, 0]).reset_index()
df.index.rename('rank', inplace=True)
df.columns.rename('', inplace=True)
BRi = df.query('sigla_provincia=="BR"').index.values[0]
BSi = df.query('sigla_provincia=="BS"').index.values[0]
display(df.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,12,BSi,BRi,-3,-2,-1], [1,-2,-1]])

In [ ]:
provs = ('BS', 'BR', 'MI', 'BA', 'RM', 'NA', 'BG')
fig, ax = plt.subplots(1,2)
plt.suptitle(f'Nuovi casi \n aggiornamento: {date_ddmmyyyy}')
for p in provs:
    m[['prov_acr']+dates[-30:-1:1]].set_index('prov_acr').T[p].plot(ax=ax[0])
    m[['prov_acr']+dates[-30:-1:1]].set_index('prov_acr').T[p].plot(logy=True, ax=ax[1])
plt.legend(provs)
plt.savefig('fig/prov.png')

fig, ax = plt.subplots(1,2)
plt.suptitle(f'Incidenza {window} giorni \n aggiornamento: {date_ddmmyyyy}')
for p in provs:
    mc[['prov_acr']+dates[-30:-1:1]].set_index('prov_acr').T[p].plot(ax=ax[0])
    mc[['prov_acr']+dates[-30:-1:1]].set_index('prov_acr').T[p].plot(logy=True, ax=ax[1])
plt.legend(provs)
plt.savefig('fig/prov_inc.png')

In [ ]:
regs = ('lombardia', 'puglia', 'emilia-romagna', 'marche', 'lazio', 'campania')
fig, ax = plt.subplots()
plt.suptitle(f'Incidenza {window} giorni \n aggiornamento: {date_ddmmyyyy}')
for p in regs:
    mcr.reset_index()[['reg_name']+dates[-30:-1:1]].set_index('reg_name').T[p].plot(ax=ax)
plt.legend(regs)
plt.savefig('fig/reg_inc.png')

fig, ax = plt.subplots()
plt.suptitle(f'Occupamento (%) Terapie Intensive: {date_ddmmyyyy}')
for p in regs:
    rTI[['reg_name']+dates[-30:-1:1]].set_index('reg_name').T[p].plot(ax=ax)
plt.legend(regs)
plt.savefig('fig/ti.png')

# Regioni

In [ ]:
reg_dir = data_dir /  'dati-regioni'
df = []
prec = lambda s: f'{s:.4f}'
for f in sorted(reg_dir.glob('dpc-covid19-ita-regioni-202*.csv')):
    tmp_df = pd.read_csv(f, encoding='latin-1')
    df.append(tmp_df)
    del tmp_df
DF = pd.concat(df)

In [ ]:
morti = DF.pivot_table(index='denominazione_regione', columns='data', values='deceduti').sum().to_numpy()
np.diff(morti)

In [ ]:
casi = DF.pivot_table(index='denominazione_regione', columns='data', values='totale_casi').sum()
np.diff(casi)

In [ ]:
plt.figure()
plt.semilogy(np.diff(casi))
plt.title('Incremento giornaliero')

plt.figure()
plt.plot(np.diff(casi))
plt.title('Incremento giornaliero')

# Dati Johns Hopkins
Data from [github.com/CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
# df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
# df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [ ]:
df.columns

In [ ]:
countries = ['Italy','France','Germany','Spain','Belgium', 'US', 'Denmark', 'Netherlands']
jhdates = [c for c in df.columns if '/2' in c]
df.groupby('Country/Region').sum().loc[countries][jhdates[-50:]].T.plot(logy=True, title='Confirmed Cases')

# Andamento Globale

In [ ]:
df = df.groupby('Country/Region').sum().loc[countries][jhdates[-10:]]
df['increment'] = df.iloc[:, -1] - df.iloc[:, -2]
df = df.sort_values(['increment', df.columns[-1]], ascending=[0, 0])
display(df)

# France

In [ ]:
FR = pd.read_csv('https://raw.githubusercontent.com/opencovid19-fr/data/master/dist/chiffres-cles.csv')

In [ ]:
idx = ['DEP' in _ for _ in FR['maille_code']]
DEP = FR.loc[idx].pivot_table(index=['maille_code', 'maille_nom'], columns=['date'], values=['cas_confirmes']).droplevel(level=0, axis=1)
DEP.sum(1).sort_values(ascending=False).to_frame().iloc[:25]

In [ ]:
FR.columns.unique()